In [ ]:
# SUP GUYS IT'S YA GIRL LET'S DO THIS SHIT
# here I am importing things 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap  #Tom  doesn't have this. How do you install it?

%matplotlib inline

In [ ]:
test = pd.read_csv('./assets/test.csv')
train = pd.read_csv('./assets/train.csv')
spray = pd.read_csv('./assets/spray.csv')
weather = pd.read_csv('./assets/weather.csv')

In [ ]:
# checking out the first five rows of the test data

test.head()

In [ ]:
train.head()

## Doing some initial analysis and cleaning on 'train' and 'test'

In [ ]:
# dropping the additional index column 

test.drop(columns = 'Id', inplace = True)

In [ ]:
# changing the 'Date' column to a datetime

train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [ ]:
# changing the date to the index

train = train.set_index('Date')
test = test.set_index('Date')

In [ ]:
# confirming it worked
print(type(train.index[0]))
train.head()

In [ ]:
# wanted to see how the counts of mosquitos and the infected mosquito counts
# related

train.sort_values('WnvPresent', ascending = False).head(20)

In [ ]:
# created an additional column that shows the percentage of infected mosquitos

train['PercentageWnv'] = train['WnvPresent'] / train['NumMosquitos']

In [ ]:
# make sure it works
train['PercentageWnv'].value_counts()

In [ ]:
train.head()

### Now to the "Spray" data

In [ ]:
spray.head()

In [ ]:
spray.isnull().sum()

In [ ]:
# decided to put the most common time in the null time values

spray['Time'].value_counts()

In [ ]:
# fill in na values with the most common time

spray['Time'].fillna('7:44:32 PM', inplace = True)

In [ ]:
# combine date and time columns 

spray['DateTime'] = spray['Date'] + ' ' + spray['Time']

In [ ]:
# convert to datetime and convert to index

spray['DateTime'] = pd.to_datetime(spray['DateTime'])
spray = spray.set_index('DateTime')

In [ ]:
# drop date and time columns as they are redundant 

spray.drop(columns = ['Date', 'Time'], inplace = True)

In [ ]:
# check to make sure it worked

spray.head()

In [ ]:
# THIS IS A TEST AND IS NOT GOOD, TK WAS WORKING ON BASEMAP AS WELL. LEAVING THIS IN INCASE ANYONE
# WANTS TO WORK ON IT

# plt.figure(figsize=(8, 8))
# earth = Basemap(projection = 'merc',
#                 lon_0 = -87.6298,
#                 lat_0 = 41.8781,
#                 llcrnrlon=-85, 
#                 llcrnrlat=39,
#                 urcrnrlon=-89, 
#                 urcrnrlat=45,
#                 resolution = 'h'
#                )
# earth.bluemarble(alpha=0.42)
# earth.drawcoastlines(color='grey', linewidth=1)
# #earth.drawcounties(color = 'grey', linewidth=1)
# plt.scatter(spray['Longitude'], spray['Latitude'], 
#             c='red',alpha=.5)
# #plt.figsize()
# plt.xlabel("Test");

### moving to 'Weather' data

In [ ]:
weather.head()

In [ ]:
weather.info()

In [ ]:
# converting date to datetime

weather['Date'] = pd.to_datetime(weather['Date'])

In [ ]:
# setting date to index

weather = weather.set_index('Date')

In [ ]:
# station 2 sucks at recording things, it has a lot more missing data.

weather.head()

In [ ]:
# here we will replace 'M' with nan values (as it is missing) and change 'T' to a small decimal (as it stands for trace)
# as it was recorded as 'trace'

weather.replace('M', np.nan, inplace = True)
weather.replace('  T', 0.005, inplace = True) #This was annoying

In [ ]:
weather.head(10) # Checking to make sure it worked. There are a few NANs and .005s in there.

In [ ]:
weather.isnull().sum()

In [ ]:
# deleting the columns that give little to no information

weather.drop(columns = ['Water1', 'Depth', 'SnowFall', 'Depart', 'CodeSum'], inplace = True)

In [ ]:
# here we are filling in the missing 'Tavg' data with the average 

weather['Tavg'].fillna(((weather['Tmax'] + weather['Tmin']) / 2), inplace = True)

In [ ]:
# making the columns with null values easier to read
# What null columns are left to deal with

null_columns = weather.columns[weather.isnull().any()]
weather[null_columns].isnull().sum()

In [ ]:
print('Null Rows: ', weather[weather.isnull().any(axis=1)][null_columns].shape[0])
print('Total Rows: ', weather.shape[0])
#Tom G thinks we should just drop these

In [ ]:
#Being overly cautious before dropping data
weather.dropna().shape

In [ ]:
weather.dropna(inplace=True)

In [ ]:
weather.drop(columns = ['Heat', 'Cool'], inplace = True)

In [ ]:
weather.head(2)

## Set the '-' from weather station 2 to the times from weather station 1


In [ ]:
print(weather[weather['Station']==1].shape[0])
print(weather[weather['Station']==2].shape[0]) #This is annoying. I was hoping that there would be the same amount in both so I could change the sunrise/sunset for station 2 easier.

In [ ]:
# There is definitely a prettier way of doing this. 
# Filling in t
for i in range(weather.shape[0]-1):
    if weather.index[i] == weather.index[i+1]:
        if weather.iloc[i, 0] == 1: #Station 1
            if weather.iloc[i+1, 0] == 2: #Station 2
                weather.iloc[i+1, 8:10] = weather.iloc[i, 8:10]
            

In [ ]:
weather.head(6) #checking to make sure that they worked correctly

In [ ]:
ind_dash = weather[weather['Sunrise']=='-'].index
weather.loc[ind_dash, :] 
#These 7 rows do not have metrics from station 1 to change their Sunrise/Sunset to

In [ ]:
weather.loc['2009-06-18', :] 
#Checking to make sure that there aren't any other data points for the dates listed above.

There are only 7, Tom G is okay with dropping them

In [ ]:
print(weather.shape[0]-len(ind_dash))

In [ ]:
weather.drop(index=ind_dash).shape

In [ ]:
weather.drop(index=ind_dash, inplace=True)

## Setting the columns to their correct types


In [ ]:
train.dtypes

In [ ]:
train.head(1) # These dtypes look right

In [ ]:
test.dtypes #these types look right

In [ ]:
weather.dtypes

In [ ]:

weather.head()

In [ ]:
objs = []
for col in weather.columns:
    if weather[col].dtypes == 'O':
        objs.append(col)
        
weather[objs].dtypes

In [ ]:
floats = ['PrecipTotal', 'StnPressure', 'SeaLevel', 'AvgSpeed']
ints = objs[0:6]
weather[floats] = weather[floats].astype(float)
weather[ints] = weather[ints].astype(int)
#print(floats)
#print(ints)

In [ ]:
weather.dtypes

Spray

In [ ]:
spray.dtypes

In [ ]:
test.isnull().sum().sum()

In [ ]:
train.isnull().sum().sum()

In [ ]:
weather.isnull().sum().sum()

In [ ]:
spray.isnull().sum().sum()

### Splitting up the two weather stations just in case we want to use them individually later

In [ ]:
weather_station_1 = weather.loc[weather['Station'] == 1]
weather_station_2 = weather.loc[weather['Station'] == 2]

In [ ]:
weather_station_1.shape[0] + weather_station_2.shape[0] == weather.shape[0]

## Save
Save the cleaned files as new csvs to use in other notebooks.

In [ ]:
train.to_csv('./assets/train_cleaned.csv')
test.to_csv('./assets/test_cleaned.csv')
spray.to_csv('./assets/spray_cleaned.csv')
weather.to_csv('./assets/weather_cleaned.csv')
weather_station_1.to_csv('./assets/weather_station_1_cleaned.csv')
weather_station_2.to_csv('./assets/weather_station_2_cleaned.csv')

# Start of Modeling

In [1]:
import pandas as pd, numpy as np, seaborn as sns

In [2]:
%pwd

'/Users/tomkelly/Desktop/general_assembly/DSI-US-5/project-4/tk'

In [3]:
train = pd.read_csv('../assets/train_cleaned.csv')
test = pd.read_csv('../assets/test_cleaned.csv')
spray = pd.read_csv('../assets/spray_cleaned.csv')
weather = pd.read_csv('../assets/weather_cleaned.csv')
weather_station_1 = pd.read_csv('../assets/weather_station_1_cleaned.csv')
weather_station_2 = pd.read_csv('../assets/weather_station_2_cleaned.csv')

In [8]:
pd.value_counts(train['Species']) 

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

In [15]:
train.shape[0]

10506

In [13]:
pd.value_counts(train['Species']).sum() * train['']

10506

In [12]:
train['NumMosquitos'].sum()

135039

In [16]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing

# Load dataset 
train = pd.read_csv('../assets/train.csv')
test = pd.read_csv('../assets/test.csv')
sample = pd.read_csv('../assets/sampleSubmission.csv')
weather = pd.read_csv('../assets/weather.csv')

In [17]:
train.WnvPresent.values

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
# Get labels
labels = train.WnvPresent.values

# Not using codesum for this benchmark
weather = weather.drop('CodeSum', axis=1)

In [19]:
# Split station 1 and 2 and join horizontally
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

# replace some missing values and T with -1
weather = weather.replace('M', -1)
weather = weather.replace('-', -1)
weather = weather.replace('T', -1)
weather = weather.replace(' T', -1)
weather = weather.replace('  T', -1)

In [20]:
# Functions to extract month and day from dataset
# You can also use parse_dates of Pandas.
def create_month(x):
    return x.split('-')[1]

def create_day(x):
    return x.split('-')[2]

train['month'] = train.Date.apply(create_month)
train['day'] = train.Date.apply(create_day)
test['month'] = test.Date.apply(create_month)
test['day'] = test.Date.apply(create_day)

In [21]:
# Add integer latitude/longitude columns
train['Lat_int'] = train.Latitude.apply(int)
train['Long_int'] = train.Longitude.apply(int)
test['Lat_int'] = test.Latitude.apply(int)
test['Long_int'] = test.Longitude.apply(int)

# drop address columns
train = train.drop(['Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos'], axis = 1)
test = test.drop(['Id', 'Address', 'AddressNumberAndStreet'], axis = 1)


In [22]:
# Merge with weather data
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')
train = train.drop(['Date'], axis = 1)
test = test.drop(['Date'], axis = 1)

In [23]:
# Convert categorical data to numbers
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train['Species'].values) + list(test['Species'].values))
train['Species'] = lbl.transform(train['Species'].values)
test['Species'] = lbl.transform(test['Species'].values)

lbl.fit(list(train['Street'].values) + list(test['Street'].values))
train['Street'] = lbl.transform(train['Street'].values)
test['Street'] = lbl.transform(test['Street'].values)

lbl.fit(list(train['Trap'].values) + list(test['Trap'].values))
train['Trap'] = lbl.transform(train['Trap'].values)
test['Trap'] = lbl.transform(test['Trap'].values)

In [24]:
# drop columns with -1s
train = train.loc[:,(train != -1).any(axis=0)]
test = test.loc[:,(test != -1).any(axis=0)]

In [25]:
# Random Forest Classifier 
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=2)

In [26]:
# labels.astype('float')

In [27]:
clf.fit(train, labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
predictions = clf.predict_proba(test)[:,1]

In [29]:
# create predictions and submission file
# predictions = clf.predict_proba(test)[:,1]
sample['WnvPresent'] = predictions
sample.to_csv('beat_the_benchmark.csv', index=False)

In [ ]:
# ('Species', 0.21019096220604577),
#  ('Block', 0.088037979163343758),
#  ('Street', 0.10868135845964108),
#  ('Trap', 0.12220930367046234),
#  ('Latitude', 0.1165644250267889),
#  ('Longitude', 0.12642414323782206)

from collections import Counter
# count = pd.Series(train['Species']).map(Counter).sum()
pd.value_counts(train.Species)

In [31]:
test

Species  Block  Street  Trap   Latitude  Longitude  AddressAccuracy  \
0             2     41      36     1  41.954690 -87.800991                9   
1             3     41      36     1  41.954690 -87.800991                9   
2             1     41      36     1  41.954690 -87.800991                9   
3             4     41      36     1  41.954690 -87.800991                9   
4             6     41      36     1  41.954690 -87.800991                9   
5             5     41      36     1  41.954690 -87.800991                9   
6             7     41      36     1  41.954690 -87.800991                9   
7             0     41      36     1  41.954690 -87.800991                9   
8             2     62      30     8  41.994991 -87.769279                9   
9             3     62      30     8  41.994991 -87.769279                9   
10            1     62      30     8  41.994991 -87.769279                9   
11            4     62      30     8  41.994991 -87.769279                9   
12            6     62      30     8  41.994991 -87.769279                9   
13            5     62      30     8  41.994991 -87.769279                9   
14            7     62      30     8  41.994991 -87.769279                9   
15            0     62      30     8  41.994991 -87.769279                9   
16            2     79     120    15  41.974089 -87.824812                8   
17            3     79     120    15  41.974089 -87.824812                8   
18            1     79     120    15  41.974089 -87.824812                8   
19            4     79     120    15  41.974089 -87.824812                8   
20            6     79     120    15  41.974089 -87.824812                8   
21            5     79     120    15  41.974089 -87.824812                8   
22            7     79     120    15  41.974089 -87.824812                8   
23            0     79     120    15  41.974089 -87.824812                8   
24            2     15     138    34  41.921600 -87.666455                8   
25            3     15     138    34  41.921600 -87.666455                8   
26            1     15     138    34  41.921600 -87.666455                8   
27            4     15     138    34  41.921600 -87.666455                8   
28            6     15     138    34  41.921600 -87.666455                8   
29            5     15     138    34  41.921600 -87.666455                8   
...         ...    ...     ...   ...        ...        ...              ...   
116263        1     34     109   146  41.753391 -87.707394                8   
116264        4     34     109   146  41.753391 -87.707394                8   
116265        6     34     109   146  41.753391 -87.707394                8   
116266        5     34     109   146  41.753391 -87.707394                8   
116267        7     34     109   146  41.753391 -87.707394                8   
116268        0     34     109   146  41.753391 -87.707394                8   
116269        2     51     103    48  41.777689 -87.749149                9   
116270        3     51     103    48  41.777689 -87.749149                9   
116271        1     51     103    48  41.777689 -87.749149                9   
116272        4     51     103    48  41.777689 -87.749149                9   
116273        6     51     103    48  41.777689 -87.749149                9   
116274        5     51     103    48  41.777689 -87.749149                9   
116275        7     51     103    48  41.777689 -87.749149                9   
116276        0     51     103    48  41.777689 -87.749149                9   
116277        2     96      73    78  41.719140 -87.669539                9   
116278        3     96      73    78  41.719140 -87.669539                9   
116279        1     96      73    78  41.719140 -87.669539                9   
116280        4     96      73    78  41.719140 -87.669539                9   
116281        6     96      73    78  41.719140 -87.669539

In [30]:
from sklearn.metrics import roc_auc_score, roc_curve
roc_auc = roc_auc_score(test, predictions)
roc_auc

TypeError: '<' not supported between instances of 'str' and 'int'